# Statystyka matematyczna - ćwiczenia laboratoryjne 2023/2024

Ten notatnik zalicza się do grupy zestawów zadań, na podstawie których odbywa się zaliczenie ćwiczeń i podlega zwrotowi do oceny w ustalonym na zajęciach terminie.

Uwagi i wytyczne ogólne dotyczące uzupełniania notatnika:
- Podczas wykonywania zadań należy korzystać wyłącznie z pakietów zaimportowanych na początku notatnika oraz z pakietów wchodzących w skład standardowej biblioteki Pythona, które można zaimportować samodzielnie we wskazanej komórce notatnika.
- Swoje rozwiązania należy wprowadzać wyłącznie w miejce następujących fragmentów kodu:<br/> `# YOUR CODE HERE`<br/> `raise NotImplementedError()`<br/> 
a odpowiedzi tekstowe w komórkach oznaczonych hasłem:<br/> 
`YOUR ANSWER HERE`<br/> 
Nie należy w żaden sposób modyfikować pozostałych fragmentów kodu oraz innych elementów notatnika, w szczególności dodawać lub usuwać komórek oraz zmieniać nazwy pliku.
- Otrzymywane wyniki i odpowiedzi mają być rezultatem wykonania napisanego kodu.
- Zadanie należy wykonać w taki sposób, aby podczas wykonywania kodu nie zostały wyświetlone żadne ostrzeżenia.
- Zawarte w notatniku automatyczne testy mają charakter poglądowy. Dotyczą one wybranych aspektów zadań i mają za zadanie wyłapać podstawowe błędy. Przejście wszystkich testów nie oznacza, że zadanie jest wykonane w całości poprawnie.

Uwagi i wytyczne ogólne dotyczące wizualizacji wyników:
- Wszystkie wykresy powinny być wykonane w jednolitym, przejrzystym i czytelnym stylu, posiadać odpowiednio dobrane proporcje i zakresy wartości osi.
- Wykresy oraz ich osie powinny mieć nadane tytuły. Jeżeli w obrębie figury znajduje się więcej niż jeden wykres to figura również powinna mieć nadany tytuł. 
- Figury powinny mieć ustawione białe tło, tak, aby niezależnie od ustawień notatnika wszystkie elementy wykresów były dobrze widoczne (domyślnie tło jest przeźroczyste co może powodować problemy w notatnikach z ustawionym ciemnym tłem).
- Rozmiar poziomy figur nie powinien przekraczać 20 cali.

Przed odesłaniem zestawu zadań do oceny proszę uzupełnić komórkę z danymi autorów rozwiązania (nazwa zespołu oraz imiona, nazwiska i numery indeksów członków zespołu) oraz upewnić się, że notatnik działa zgodnie z oczekiwaniami. W tym celu należy skorzystać z opcji **Restart Kernel and Run All Cells...** dostępnej na górnej belce notatnika pod symbolem $\blacktriangleright\blacktriangleright$.

Nazwa zespołu: 2.38 Członkowie: Roksana Jandura , Katarzyna Wesołwoska , Katarzyna Tokarczuk , Magdalena Pogorzelec 

---

# Zestaw zadań 11: Weryfikacja hipotez statystycznych - metoda bootstap i testy permutacyjne

In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

### Dane do zadań

W celu wygenerowania danych wykorzystywanych w zawartych w notatniku zadaniach i komórkach testowych wykonaj obie poniższe komórki.

In [2]:
test_data_1 = pd.DataFrame(data=sp.stats.norm.rvs(loc=5, scale=0.2, size=15, random_state=7), columns=["X"])
test_data_1.head()

,X
0,5.338105
1,4.906813
2,5.006564
3,5.081503
4,4.842215


In [3]:
test_data_2 = pd.DataFrame(data=sp.stats.norm.rvs(loc=5.12, scale=0.25, size=23, random_state=19), columns=["X"])
test_data_2.head()

,X
0,5.175251
1,5.034884
2,4.975563
3,5.018992
4,4.969178


### Zadanie 1: Weryfikacja hipotezy o wartości oczekiwanej metodą bootstrap [7 pkt]

Przygotuj funkcję `bootstrap_mean_NHST()`, która będzie weryfikować hipotezę o wartości przeciętnej metodą bootstrap i wyświetlać wynik testu statystycznego zgodnie z wytycznymi zawartymi w dołączonym do notatnika zestawie wzorów, w treści zadania oraz w docstring funkcji.

Uwagi do wykonania zadania:
 - Automatyczne testy zakładają losowanie prób wg następującego schematu - kolejne próby są losowane w pętli for, a funkcja losująca próbę (np. `pd.sample()`) w kolejnych losowaniach ma ustawioną wartość parametru odpowiadającego za ziarno generatora liczb losowych na wartość `random_state`, `random_state+1`, `random_state+2` itd. 
 - Celem zadania jest napisanie funkcji od podstaw, w rozwiązaniu nie należy korzystać z gotowych funkcji służących do estymacji bootstrapowej.

In [4]:
def bootstrap_mean_NHST(data, H0, H1, number_of_samples, alpha, random_state):
    """
    Parameters
    -------
    data: DataFrame
        Tablica zawierająca domyślny indeks i kolumnę "X" wynikami próby losowej.
    H0: float
        Wartość przeciętna przyjęta jako hipoteza zerowa.
    H1: str
        Postać hipotezy alternatywnej, przyjmuje wartości:
        - two-sided: wartość przeciętna jest różna od wartości przyjętej w H0,
        - less: wartość przeciętna jest mniejsza od wartości przyjętej w H0,
        - greater: wartość przeciętna jest większa od wartości przyjętej w H0.
    alpha: float
        Wartość określająca poziom istotności.
    
    Returns
    -------
    pvalue: float
        Prawdopodobieństwo otrzymania bardziej ekstremalnej wartości statystyki testowej (w kierunku hipotezy alternatywnej)
        względem wartości otrzymanej na podstawie próby losowej.
    H: int
        Wynik testu statystycznego, przyjmuje wartość:
        0 - gdy wynik testu istotności nie daje podstaw do odrzucenia H0 na rzecz H1 na poziomie istotności 1-alpha,
        1 - gdy następuje odrzucenie H0 na rzecz H1 na poziomie istotności 1-alpha.
    """
    # YOUR CODE HERE

    #populacja bootstrapowa
    sample_mean = data['X'].mean()
    bootstrap_population = data['X'] + (H0 - sample_mean)
    
    n = len(data)
    bootstrap_samples = np.empty((number_of_samples, n))

    #losowanie prób bootstrapowych
    for i in range(number_of_samples):
        np.random.seed(random_state + i)
        bootstrap_samples[i] = np.random.choice(bootstrap_population, size=n, replace=True)
    
    bootstrap_means = bootstrap_samples.mean(axis=1)
    
    #wyliczanie pvalue
    if H1 == 'two-sided':
        pvalue = (np.mean(np.abs(bootstrap_means - H0) >= np.abs(sample_mean - H0)))
    elif H1 == 'less':
        pvalue = (np.mean(bootstrap_means <= sample_mean))
    elif H1 == 'greater':
        pvalue = (np.mean(bootstrap_means >= sample_mean))

    #decyzja weryfikująca
    H = 1 if pvalue < alpha else 0
    print(f"H wynosi {H:} dla wartosci pvalue {pvalue:}.")
    return pvalue, H

In [5]:
# Komórka testowa
assert np.all(np.isclose(bootstrap_mean_NHST(test_data_1, 5.08, "two-sided", 1000, 0.05, 10), (0.055, 0)))
assert np.all(np.isclose(bootstrap_mean_NHST(test_data_1, 5.08, "less", 2000, 0.05, 12), (0.025, 1)))
assert np.all(np.isclose(bootstrap_mean_NHST(test_data_1, 5.08, "greater", 1500, 0.1, 15), (0.974, 0)))

H wynosi 0 dla wartosci pvalue 0.055.
H wynosi 1 dla wartosci pvalue 0.025.
H wynosi 0 dla wartosci pvalue 0.974.


### Zadanie 2: Weryfikacja hipotezy o dwóch wartościach oczekiwanych metodą bootstrap [7 pkt]

Przygotuj funkcję `bootstrap_means_NHST()`, która będzie weryfikować hipotezę o dwóch wartościach przeciętnych metodą bootstrap i wyświetlać wynik testu statystycznego zgodnie z wytycznymi zawartymi w dołączonym do notatnika zestawie wzorów, w treści zadania oraz w docstring funkcji.

Uwagi do wykonania zadania:
 - Automatyczne testy zakładają losowanie prób wg następującego schematu - kolejne próby są losowane w pętli for, a funkcja losująca próbę (np. `pd.sample()`) w kolejnych losowaniach ma ustawioną wartość parametru odpowiadającego za ziarno generatora liczb losowych na wartość `random_state` `random_state+2`, `random_state+4` itd. (pierwsza próba) oraz `random_state+1` `random_state+3`, `random_state+5` itd. (druga próba).
 - Celem zadania jest napisanie funkcji od podstaw, w rozwiązaniu nie należy korzystać z gotowych funkcji służących do estymacji bootstrapowej.

In [6]:
def bootstrap_means_NHST(data1, data2, H1, number_of_samples, alpha, random_state):
    """
    Parameters
    -------
    data1: DataFrame
        Tablica zawierająca domyślny indeks i kolumnę "X" wynikami pierwszej próby losowej.
    data2: DataFrame
        Tablica zawierająca domyślny indeks i kolumnę "X" wynikami drugiej próby losowej.
    H1: str
        Postać hipotezy alternatywnej, przyjmuje wartości:
        - two-sided: wartość przeciętna populacji, z których zostały pobrane próby jest różna,
        - less: wartość przeciętna populacji, z której została pobrana druga próba jest mniejsza,
        - greater: wartość przeciętna populacji, z której została pobrana druga próba jest większa,
    alpha: float
        Wartość określająca poziom istotności.
    
    Returns
    -------
    pvalue: float
        Prawdopodobieństwo otrzymania bardziej ekstremalnej wartości statystyki testowej (w kierunku hipotezy alternatywnej)
        względem wartości otrzymanej na podstawie próby losowej.
    H: int
        Wynik testu statystycznego, przyjmuje wartość:
        0 - gdy wynik testu istotności nie daje podstaw do odrzucenia H0 na rzecz H1 na poziomie istotności 1-alpha,
        1 - gdy następuje odrzucenie H0 na rzecz H1 na poziomie istotności 1-alpha.
    """
    # YOUR CODE HERE
    #raise NotImplementedError()
    
    # Rozmiary prób
    n1 = len(data1)
    n2 = len(data2)
    
    # Obliczenie oryginalnej różnicy średnich
    original_mean_diff = data1['X'].mean() - data2['X'].mean()
    
    # Połączenie obu prób w jedną populację bootstrapową
    n = np.concatenate([data1['X'].values, data2['X'].values])
    
    # Lista do przechowywania różnic średnich z prób bootstrapowych
    bootstrap_diffs = []
    
    # Losowanie prób bootstrapowych
    for i in range(number_of_samples):
        # Losowanie prób z populacji bootstrapowej
        np.random.seed(random_state + 2 * i)
        sample1 = np.random.choice(n, n1, replace=True)
        
        np.random.seed(random_state + 2 * i + 1)
        sample2 = np.random.choice(n, n2, replace=True)
        
        # Obliczenie różnicy średnich dla prób bootstrapowych
        mean_diff = sample1.mean() - sample2.mean()
        bootstrap_diffs.append(mean_diff)
    
    bootstrap_diffs = np.array(bootstrap_diffs)
    
    # Obliczenie p-value w zależności od hipotezy alternatywnej
    if H1 == 'two-sided':
        pvalue = np.mean(np.abs(bootstrap_diffs) >= np.abs(original_mean_diff))
    elif H1 == 'greater':
        pvalue = np.mean(bootstrap_diffs <= original_mean_diff)
    elif H1 == 'less':
        pvalue = np.mean(bootstrap_diffs >= original_mean_diff)
    
    # Decyzja o odrzuceniu H0 na rzecz H1
    H = int(pvalue <= alpha)
    print(f"H wynosi {H:} dla wartosci pvalue {pvalue:}.")
    return pvalue, H

In [7]:
# Komórka testowa
assert np.all(np.isclose(bootstrap_means_NHST(test_data_1, test_data_2, "two-sided", 3000, 0.05, 144), (0.091, 0)))
assert np.all(np.isclose(bootstrap_means_NHST(test_data_1, test_data_2, "less", 1000, 0.1, 10), (0.954, 0)))
assert np.all(np.isclose(bootstrap_means_NHST(test_data_1, test_data_2, "greater", 1500, 0.05, 132), (0.046, 1)))

H wynosi 0 dla wartosci pvalue 0.091.
H wynosi 0 dla wartosci pvalue 0.954.
H wynosi 1 dla wartosci pvalue 0.046.


### Zadanie 3: Weryfikacja hipotezy o dwóch wartościach oczekiwanych metodą testu permutacyjnego [7 pkt]

Przygotuj funkcję `permutation_means_NHST()`, która będzie weryfikować hipotezę o dwóch wartościach przeciętnych metodą testu permutacyjnego i wyświetlać wynik testu statystycznego zgodnie z wytycznymi zawartymi w dołączonym do notatnika zestawie wzorów, w treści zadania oraz w docstring funkcji.

Uwagi do wykonania zadania:
 - Automatyczne testy zakładają losowanie prób wg następującego schematu - kolejne próby są losowane w pętli for, a funkcja losująca pierwszą próbę (np. `pd.sample()`) w kolejnych losowaniach ma ustawioną wartość parametru odpowiadającego za ziarno generatora liczb losowych na wartość `random_state`, `random_state+1`, `random_state+2` itd. Pozostałe obserwacje trafiają do drugiej próby.
 - Celem zadania jest napisanie funkcji od podstaw, w rozwiązaniu nie należy korzystać z gotowych funkcji służących do estymacji bootstrapowej.

In [8]:
def permutation_means_NHST(data1, data2, H1, number_of_samples, alpha, random_state):
    """
    Parameters
    -------
    data1: DataFrame
        Tablica zawierająca domyślny indeks i kolumnę "X" wynikami pierwszej próby losowej.
    data2: DataFrame
        Tablica zawierająca domyślny indeks i kolumnę "X" wynikami drugiej próby losowej.
    H1: str
        Postać hipotezy alternatywnej, przyjmuje wartości:
        - two-sided: wartości przeciętne populacji, z których zostały pobrane próby są różne,
        - less: wartość przeciętna populacji, z której została pobrana druga próba jest mniejsza,
        - greater: wartość przeciętna populacji, z której została pobrana druga próba jest większa,
    alpha: float
        Wartość określająca poziom istotności.
    
    Returns
    -------
    pvalue: float
        Prawdopodobieństwo otrzymania bardziej ekstremalnej wartości statystyki testowej (w kierunku hipotezy alternatywnej)
        względem wartości otrzymanej na podstawie próby losowej.
    H: int
        Wynik testu statystycznego, przyjmuje wartość:
        0 - gdy wynik testu istotności nie daje podstaw do odrzucenia H0 na rzecz H1 na poziomie istotności 1-alpha,
        1 - gdy następuje odrzucenie H0 na rzecz H1 na poziomie istotności 1-alpha.
    """
    #łączenie danych
    dane = pd.concat([data1['X'], data2['X']])
    n1 = len(data1) #dlugosci danych
    n2 = len(data2)
    
    #różnica średnich (wybór odpowiedniej statystyki testowej)
    srednia_obserwowana=data2['X'].mean()-data1['X'].mean()

    #pusta tablica do przechowywania roznic sredniej
    srednia=np.zeros(number_of_samples)
    for i in range(number_of_samples):
        mieszanie=dane.sample(frac=1, replace=False, random_state=random_state + i) #pomieszanie danych
        x1= mieszanie.iloc[:n1] #podział na dwie części
        x2= mieszanie.iloc[n1:]
        srednia[i] = x2.mean() - x1.mean() #różnica średnich między tymi częściami

    #pvalue na podstawie hipotezy alternatywnej
    if H1 == 'two-sided':
        pvalue = np.mean(np.abs(srednia)>=np.abs(srednia_obserwowana)) #zarówna większa jak i mniejsza
    elif H1 == 'greater':
        pvalue = np.mean(srednia>=srednia_obserwowana)
    elif H1 == 'less':
        pvalue = np.mean(srednia<=srednia_obserwowana)
    
    #decyzja o odrzuceniu H0 na podstawie H1
    if  pvalue < alpha:
        H=1
    else:
        H=0

    print(f"H wynosi {H:} dla wartosci pvalue {pvalue:}.")
    return pvalue, H

In [9]:
# Komórka testowa
assert np.all(np.isclose(permutation_means_NHST(test_data_1, test_data_2, "two-sided", 2000, 0.1, 67), (0.0905, 1)))
assert np.all(np.isclose(permutation_means_NHST(test_data_1, test_data_2, "less", 1000, 0.05, 15), (0.945, 0)))
assert np.all(np.isclose(permutation_means_NHST(test_data_1, test_data_2, "greater", 2000, 0.05, 12), (0.0465, 1)))

H wynosi 1 dla wartosci pvalue 0.0905.
H wynosi 0 dla wartosci pvalue 0.945.
H wynosi 1 dla wartosci pvalue 0.0465.


### Zadanie 4: Weryfikacja hipotezy o wartości oczekiwanej metodą testu permutacyjnego (dedykowana funkcja) [4 pkt]

#### a) 
Wykorzystaj funkcję `scipy.stats.permutation_test()`, do zwertfikowania hipotezy o równości wartościach przeciętnych populacji z których zostały pobrane próby `test_data_1` i `test_data_2` względem hipotezy alternatywnej, że wartości oczekiwane tych populacji są różne. W obliczeniach przyjmij poziom istotności 1 - alpha = 0.9.

Uwagi do wykonania zadania:
 - Podczas obliczeń ustaw wartość argumentu funkcji `scipy.stats.permutation_test()` odpowiadającego za ziarno generatora liczb losowych na 29.

In [10]:
# YOUR CODE HERE
def statistic_1(x, y):
    return np.mean(x)-np.mean(y)

#wartości oczekiwane mają być są różne.
result_1 = sp.stats.permutation_test((test_data_1, test_data_2), statistic=statistic_1, random_state=29, alternative='two-sided')

pvalue_1 = result_1.pvalue
#1-alpha=0.9 czyli alpha=0.1
alpfa=0.1
if pvalue_1 <= alpfa:
    H_1=1 #odrzucamy hipoteze zerową
else:
    H_1=0 #nie odrzucamy hipotezy zerowej


In [11]:
# Komórka testowa
assert np.isclose(pvalue_1, 0.0944)
assert H_1 == 1

#### b) 
Wykorzystaj funkcję `scipy.stats.permutation_test()`, do zwertfikowania hipotezy o równości wariancji populacji z których zostały pobrane próby `test_data_1` i `test_data_2` względem hipotezy alternatywnej, że wariacja populacji, z której została pobrana próba `test_data_2` jest wyższa. W obliczeniach przyjmij poziom istotności 1 - alpha = 0.9.

Uwagi do wykonania zadania:
 - Automatyczne testy przewidują obliczenie wartości wariancji dla wartości parametru `ddof=1`.
 - Podczas obliczeń ustaw wartość argumentu funkcji `scipy.stats.permutation_test()` odpowiadającego za ziarno generatora liczb losowych na 29.

In [12]:
# YOUR CODE HERE
def statistic_2(x, y):
    return np.var(x, ddof=1)-np.var(y, ddof=1)

#wariacja populacji z test_data_2 powinna być wyższa
result_2 = sp.stats.permutation_test((test_data_1, test_data_2), statistic=statistic_2, random_state=29,alternative='less')

pvalue_2 = result_2.pvalue
#1-alpha=0.9 czyli alpha=0.1
alpfa=0.1
if pvalue_2 <= alpfa:
    H_2=1 #odrzucamy hipoteze zerową
else:
    H_2=0 #nie odrzucamy hipotezy zerowej


In [13]:
# Komórka testowa
assert np.isclose(pvalue_2, 0.1139)
assert H_2 == 0